In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline
tf.config.set_visible_devices([], 'GPU')

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder, image_size=(150, 150)):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, image_size)
            img_data = np.asarray(img)
            images.append(img_data)
    return images

In [3]:
def load_images_from_folder_2(folder, image_size=(150, 150)):
    images = []
    for filename in os.listdir(folder):
        loaded_img = load_img(os.path.join(folder,filename), target_size=image_size)
        numpy_img = img_to_array(loaded_img)
        images.append(numpy_img)
    return images

In [4]:
# input_image_dim = (150, 150, 3)
# img_size = (150, 150)
# covid_folder = 'CT_NonCOVID'
# non_covid_folder = 'CT_COVID'

input_image_dim = (250, 250, 3)
img_size = (250, 250)
covid_folder = 'CT_NonCOVID_large'
non_covid_folder = 'CT_COVID_large'

In [5]:
control_images = load_images_from_folder_2(covid_folder, image_size=img_size)
covid_images = load_images_from_folder_2(non_covid_folder, image_size=img_size)
all_images = control_images + covid_images
all_images = np.array(all_images)
# all_images = all_images.reshape((-1, 150, 150, 3))

In [6]:

all_images.shape

(591, 250, 250, 3)

In [7]:
# for i in range(4):
#     plt.subplot(2, 2, i + 1)
#     plt.imshow(np.uint8(all_images[i]))
# plt.show

In [8]:
control_labels = [0] * len(control_images)
covid_labels = [1] * len(covid_images)
label_series = pd.Series(control_labels + covid_labels)
print(len(all_images))
print(len(label_series))

591
591


In [9]:
X_train, X_test, y_train, y_test = train_test_split(all_images, label_series, test_size=0.25, random_state=42)

In [10]:
print(len(X_train))
print(len(y_train))

443
443


In [11]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=input_image_dim)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, batch_size=1, epochs=7, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Simple network 1'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 443 samples
Epoch 1/7
443/443 [==============================] - 117s 265ms/sample - loss: 10.7547 - accuracy: 0.6411
Epoch 2/7
443/443 [==============================] - 116s 261ms/sample - loss: 0.5156 - accuracy: 0.8307
Epoch 3/7
443/443 [==============================] - 113s 255ms/sample - loss: 0.4008 - accuracy: 0.9052
Epoch 4/7
443/443 [==============================] - 117s 264ms/sample - loss: 0.5181 - accuracy: 0.8781
Epoch 5/7
443/443 [==============================] - 117s 265ms/sample - loss: 0.2086 - accuracy: 0.9300
Epoch 6/7
443/443 [==============================] - 119s 268ms/sample - loss: 0.1427 - accuracy: 0.9526
Epoch 7/7
443/443 [==============================] - 117s 263ms/sample - loss: 0.4042 - accuracy: 0.8962
Simple network 1 - accuracy: 0.7162162
Simple network 1 - recall: 0.44642857
Simple network 1 - precision: 0.6944444
Simple network 1 - F1 score: 0.54347825405503


In [13]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=input_image_dim)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, batch_size=1, epochs=7, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Simple network 2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 443 samples
Epoch 1/7
443/443 [==============================] - 147s 331ms/sample - loss: 1.4875 - accuracy: 0.6208
Epoch 2/7
443/443 [==============================] - 147s 332ms/sample - loss: 0.5341 - accuracy: 0.7336
Epoch 3/7
443/443 [==============================] - 147s 331ms/sample - loss: 0.4067 - accuracy: 0.8036
Epoch 4/7
443/443 [==============================] - 147s 332ms/sample - loss: 0.5162 - accuracy: 0.8036
Epoch 5/7
443/443 [==============================] - 147s 331ms/sample - loss: 0.4209 - accuracy: 0.8284
Epoch 6/7
443/443 [==============================] - 147s 331ms/sample - loss: 0.3508 - accuracy: 0.8600
Epoch 7/7
443/443 [==============================] - 146s 330ms/sample - loss: 0.2384 - accuracy: 0.9120
Simple network 2 - accuracy: 0.6959459
Simple network 2 - recall: 0.5714286
Simple network 2 - precision: 0.6037736
Simple network 2 - F1 score: 0.587156013556076


In [15]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.MobileNetV2(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.3)
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    dropout_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\bruce\miniconda3\envs\capstone\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [16]:
model.fit(X_train, y_train, batch_size=1, epochs=5, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Mobile Net V2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 443 samples
Epoch 1/5
443/443 [==============================] - 49s 111ms/sample - loss: 0.7039 - accuracy: 0.5124
Epoch 2/5
443/443 [==============================] - 46s 105ms/sample - loss: 0.6001 - accuracy: 0.7269
Epoch 3/5
443/443 [==============================] - 46s 105ms/sample - loss: 0.5106 - accuracy: 0.8600
Epoch 4/5
443/443 [==============================] - 46s 105ms/sample - loss: 0.4178 - accuracy: 0.9345
Epoch 5/5
443/443 [==============================] - 46s 104ms/sample - loss: 0.3202 - accuracy: 0.9842
Mobile Net V2 - accuracy: 0.5810811
Mobile Net V2 - recall: 0.23214285
Mobile Net V2 - precision: 0.40625
Mobile Net V2 - F1 score: 0.2954545348446915


In [17]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.ResNet50V2(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size=1, epochs=3, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'ResNet50 V2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 443 samples
Epoch 1/3
443/443 [==============================] - 114s 256ms/sample - loss: 0.6748 - accuracy: 0.5711
Epoch 2/3
443/443 [==============================] - 112s 254ms/sample - loss: 0.3734 - accuracy: 0.9526
Epoch 3/3
443/443 [==============================] - 112s 252ms/sample - loss: 0.0953 - accuracy: 0.9977
ResNet50 V2 - accuracy: 0.5
ResNet50 V2 - recall: 0.5535714
ResNet50 V2 - precision: 0.3875
ResNet50 V2 - F1 score: 0.45588232313552635


In [23]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00002)

base_model = tf.keras.applications.VGG16(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.3)
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dropout_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, batch_size=1, epochs=12, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'VGG16'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 443 samples
Epoch 1/12
443/443 [==============================] - 162s 365ms/sample - loss: 0.7244 - accuracy: 0.6208
Epoch 2/12
443/443 [==============================] - 176s 397ms/sample - loss: 0.5682 - accuracy: 0.7223
Epoch 3/12
443/443 [==============================] - 176s 398ms/sample - loss: 0.5021 - accuracy: 0.7652
Epoch 4/12
443/443 [==============================] - 180s 406ms/sample - loss: 0.4335 - accuracy: 0.8059
Epoch 5/12
443/443 [==============================] - 179s 404ms/sample - loss: 0.4490 - accuracy: 0.8126
Epoch 6/12
443/443 [==============================] - 180s 407ms/sample - loss: 0.4028 - accuracy: 0.8352
Epoch 7/12
443/443 [==============================] - 171s 385ms/sample - loss: 0.2561 - accuracy: 0.8939
Epoch 8/12
443/443 [==============================] - 171s 385ms/sample - loss: 0.2055 - accuracy: 0.9165
Epoch 9/12
443/443 [==============================] - 178s 401ms/sample - loss: 0.3551 - accuracy: 0.8736
Epoch 10/12
443/443 [====

In [33]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00002)

base_model = tf.keras.applications.VGG19(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.3)
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    dropout_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, batch_size=1, epochs=12, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'VGG19'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 443 samples
Epoch 1/12
443/443 [==============================] - 217s 489ms/sample - loss: 0.7511 - accuracy: 0.6072
Epoch 2/12
443/443 [==============================] - 222s 501ms/sample - loss: 0.5658 - accuracy: 0.7336
Epoch 3/12
443/443 [==============================] - 222s 501ms/sample - loss: 0.4333 - accuracy: 0.8149
Epoch 4/12
443/443 [==============================] - 222s 501ms/sample - loss: 0.4062 - accuracy: 0.8307
Epoch 5/12
443/443 [==============================] - 225s 508ms/sample - loss: 0.4442 - accuracy: 0.8081
Epoch 6/12
443/443 [==============================] - 216s 487ms/sample - loss: 0.3262 - accuracy: 0.8510
Epoch 7/12
443/443 [==============================] - 215s 484ms/sample - loss: 0.2427 - accuracy: 0.9029
Epoch 8/12
443/443 [==============================] - 214s 483ms/sample - loss: 0.2692 - accuracy: 0.8939
Epoch 9/12
443/443 [==============================] - 215s 485ms/sample - loss: 0.2491 - accuracy: 0.8962
Epoch 10/12
443/443 [====